# How-to: retrieve results from backend

In [1]:
from qiskit import transpile
from qiskit.circuit.random import random_circuit

from qiskit_braket_plugin import AWSBraketProvider

Let's create random circuit

In [4]:
circuit = random_circuit(5, 3, seed=42)
circuit.draw()

┌───┐
q_0: ─────X────────────■───────────────────────────┤ X ├
          │            │      ┌─────┐┌────────────┐└─┬─┘
q_1: ─────X────────────┼──────┤ Sdg ├┤ Rx(3.9689) ├──┼──
     ┌──────────┐      │      └─────┘└────────────┘  │  
q_2: ┤ U1(6.13) ├──────┼─────────X───────────────────■──
     └──┬───┬───┘┌─────┴─────┐   │       ┌───┐       │  
q_3: ───┤ T ├────┤ Rz(5.823) ├───┼───────┤ S ├───────┼──
       ┌┴───┴┐   └───────────┘   │       └───┘       │  
q_4: ──┤ Tdg ├───────────────────X───────────────────■──
       └─────┘

Get backend

In [5]:
provider = AWSBraketProvider()
backend = provider.get_backend("SV1")
backend

BraketBackend[SV1]

Transpile circuit against given backend

In [6]:
transpiled_circuit = transpile(circuit, backend=backend, seed_transpiler=42)
transpiled_circuit.draw(idle_wires=False, fold=-1)

global phase: 4.8274
                                                                          ┌───┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌───┐ ┌───┐      
q_0 -> 0 ───────────X────────────────────■─────────────────────────────■──┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
                    │           ┌─────┐  │        ┌────────────┐       │  └───┘└─┬─┘└─────┘└─┬─┘└───┘└─┬─┘└─────┘└─┬─┘└───┘ └───┘      
q_1 -> 1 ───────────X───────────┤ Sdg ├──┼────────┤ Rx(3.9689) ├───────┼─────────┼───────────┼─────────┼───────────┼───────────────────
         ┌─────────────────────┐└─────┘  │        └────────────┘       │         │           │         │           │        ┌───┐      
q_2 -> 2 ┤ P(6.13001602516006) ├───X─────┼─────────────────────────────┼─────────┼───────────■─────────┼───────────■────■───┤ T ├───■──
         └────┬────────────┬───┘   │   ┌─┴─┐┌───────────────────────┐┌─┴─┐┌───┐  │                     │                │   └───┘   │  
q_3 -> 3 ─────┤ P(-2.5863) ├───────┼───┤ X ├┤ Rz(-2.91151808057099) ├┤ X ├┤ S ├──┼─────────────────────┼────────────────┼───────────┼──
              └──┬─────┬───┘       │   └───┘└───────────────────────┘└───┘└───┘  │                     │   ┌───┐      ┌─┴─┐┌─────┐┌─┴─┐
q_4 -> 4 ────────┤ Tdg ├───────────X─────────────────────────────────────────────■─────────────────────■───┤ T ├──────┤ X ├┤ Tdg ├┤ X ├
                 └─────┘                                                                                   └───┘      └───┘└─────┘└───┘

Run circuit

In [7]:
job = backend.run(transpiled_circuit, shots=10)

In [8]:
f"Task id {job.job_id().split('/')[-1]}"

'Task id 3e6b6feb-fdc9-42c3-a7ba-0c4cd94089e2'

Retrieve job by id from backend

In [10]:
retrieved_job = backend.retrieve_job(job_id=job.job_id())
retrieved_job

In [12]:
retrieved_job.result().get_counts()

{'00010': 7, '00000': 3}